In [77]:
import sys
#import numpy as np
import tensorflow as tf
#from tensorflow import keras
#import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D


class myCallBack(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs['loss'] < 0.35:
            print()
            print("Loss is low so cancelling training.")
            self.model.stop_training = True

print("\nVersiones:")
print(f"- Python {sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}")
print(f"- TensorFlow {tf.__version__}")
print(f"- tf.Keras {tf.keras.__version__}")


Versiones:
- Python 3.9.23
- TensorFlow 2.18.0
- tf.Keras 3.10.0


In [78]:
# Dataset Fashion-MNIST, que está en storage.googleapis.com en TensorFlow/Keras
# Tenemos 4 matrices (tensores) organizados en dos tuplas (listas solo lectura)
# train_images: Matriz con imágenes de entrenamiento
# train_labels: Matriz con las etiquetas de las imágenes de entrenamiento
# test_images: Matriz con imágenes de prueba
# test_labels: Matriz con las etiquetas de las imágenes de prueba
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

print("Dataset (entrenamiento y test) cargado.")

Dataset (entrenamiento y test) cargado.


In [79]:
# Lista con los nombres de las clases de ropa
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [80]:
# Normalizacion de las matrices de imágenes de entrenamiento y test: int->float y [0-255] -> [0-1]
print("Normalización de entrenamiento y test: int->float y [0-255] -> [0-1]")
train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))
train_images = train_images.astype('float32')
test_images = test_images.astype('float32')
train_images /= 255
test_images /= 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

Normalización de entrenamiento y test: int->float y [0-255] -> [0-1]


In [81]:
# Best practice - comprobar la forma de los datos
print("\nDataset check:")
print("- tran_images shape: ", train_images.shape)
print("- train_labels len: ", len(train_labels))
print("- test_images shape: ", test_images.shape)
print("- test_labels len: ", len(test_labels))
print("- class_names len: ", len(class_names))
print("- train_labels: ", train_labels)

"""
plt.figure(figsize=(12,12))
for i in range (50):
    plt.subplot(10, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()
"""


Dataset check:
- tran_images shape:  (60000, 28, 28, 1)
- train_labels len:  60000
- test_images shape:  (10000, 28, 28, 1)
- test_labels len:  10000
- class_names len:  10
- train_labels:  [[0. 0. 0. ... 0. 0. 1.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


'\nplt.figure(figsize=(12,12))\nfor i in range (50):\n    plt.subplot(10, 5, i+1)\n    plt.xticks([])\n    plt.yticks([])\n    plt.grid(False)\n    plt.imshow(train_images[i], cmap=plt.cm.binary)\n    plt.xlabel(class_names[train_labels[i]])\nplt.show()\n'

In [82]:
# Definición del modelo
print("Definición del modelo")
model = Sequential()
model.add(Input(shape=(28, 28, 1)))
model.add(Conv2D(32, (5, 5), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))
model.summary()

Definición del modelo


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_10 (Conv2D)              │ (None, 24, 24, 32)     │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 12, 12, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 8, 8, 64)       │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 4, 4, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │        10,250 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 62,346 (243.54 KB)

 Trainable params: 62,346 (243.54 KB)

 Non-trainable params: 0 (0.00 B)

In [83]:
print("Compilación del modelo")
#model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=['accuracy'])
model.compile(loss="categorical_crossentropy", optimizer="sgd", metrics=['accuracy'])

Compilación del modelo


In [84]:
# Entrenamiento del modelo
print("Entrenamiento del modelo")
history = model.fit(train_images, train_labels, batch_size=100, epochs=30, callbacks=[myCallBack()])

Entrenamiento del modelo
Epoch 1/30
600/600 ━━━━━━━━━━━━━━━━━━━━ 9s 15ms/step - accuracy: 0.5079 - loss: 1.5952
Epoch 2/30
600/600 ━━━━━━━━━━━━━━━━━━━━ 11s 19ms/step - accuracy: 0.7433 - loss: 0.6999
Epoch 3/30
600/600 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.7802 - loss: 0.6019
Epoch 4/30
600/600 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.7985 - loss: 0.5570
Epoch 5/30
600/600 ━━━━━━━━━━━━━━━━━━━━ 26s 43ms/step - accuracy: 0.8156 - loss: 0.5155
Epoch 6/30
600/600 ━━━━━━━━━━━━━━━━━━━━ 24s 40ms/step - accuracy: 0.8262 - loss: 0.4846
Epoch 7/30
600/600 ━━━━━━━━━━━━━━━━━━━━ 20s 34ms/step - accuracy: 0.8354 - loss: 0.4703
Epoch 8/30
600/600 ━━━━━━━━━━━━━━━━━━━━ 20s 33ms/step - accuracy: 0.8411 - loss: 0.4526
Epoch 9/30
600/600 ━━━━━━━━━━━━━━━━━━━━ 20s 34ms/step - accuracy: 0.8490 - loss: 0.4354
Epoch 10/30
600/600 ━━━━━━━━━━━━━━━━━━━━ 21s 35ms/step - accuracy: 0.8494 - loss: 0.4246
Epoch 11/30
600/600 ━━━━━━━━━━━━━━━━━━━━ 21s 35ms/step - accuracy: 0.8573 - loss: 0.4109
Epoch 

In [85]:
#Evaluación del modelo usando los datos de Validación o Evaluación
#Son datos no se han usado para el entrenamiento
print("Evaluación:")
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {int(test_acc*100)}%")




Evaluación:
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8704 - loss: 0.3717
Test accuracy: 86%


In [86]:
# Guardo el modelo en disco
print("Saving the model")
model.save("./mnist-fashion.keras")

Saving the model


In [87]:
# Leo el modelo de disco
print("Loading the model")
model = keras.models.load_model("./mnist-fashion.keras")

Loading the model


In [88]:
# Uso del modelo
print("Predict: hago que el modelo evalúe todas las imágenes de Test")
predictions = model.predict(test_images)

print(predictions[5])
print("El modelo reconoce: ", np.argmax(predictions[5]))
print("Etiqueta correcta: ", test_labels[5])      

"""
i = 5
plt.figure(figsize=(6, 3))
plt.subplot(1, 2, 1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1, 2, 2)
plot_value_array(i, predictions[i], test_labels)
plt.show()

i = 8
plt.figure(figsize=(6, 3))
plt.subplot(1, 2, 1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1, 2, 2)
plot_value_array(i, predictions[i], test_labels)
plt.show()

num_rows = 7
num_cols = 5
num_images = num_rows * num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
    plt.subplot(num_rows, 2*num_cols, 2*i+1)
    plot_image(i, predictions[i], test_labels, test_images)
    plt.subplot(num_rows, 2*num_cols, 2*i+2)
    plot_value_array(i, predictions[i], test_labels)
plt.tight_layout()
plt.show()
"""

print("\nFin del programa.")

Predict: hago que el modelo evalúe todas las imágenes de Test
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step
[3.1025120e-05 9.9923450e-01 5.5238061e-06 6.8752590e-05 5.1596836e-04
 2.2553648e-11 7.5076559e-06 6.6400184e-12 1.3670293e-04 6.1397914e-13]
El modelo reconoce:  1
Etiqueta correcta:  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]

Fin del programa.
